# A multiscale modeling and optimization framework for the design of energy systems with embedded life cycle considerations


__author__ = "Yilun Lin"
__copyright__ = "Copyright 2023, Multi-parametric Optimization & Control Lab"
__credits__ = ["Yilun Lin", "Rahul Kakodkar", "Efstratios N. Pistikopoulos"]
__license__ = "Open"
__version__ = "1.0.0"
__maintainer__ = "Rahul Kakodkar"
__email__ = "cacodcar@tamu.edu"
__status__ = "Production"


## In the model, there are a multitude of options for:

**power generation**

**energy storage**

$\textbf{Import modules}$

In [21]:
import sys
sys.path.append('../../src')

In [22]:

import pandas 
import numpy
from energiapy.components.temporal_scale import TemporalScale
from energiapy.components.resource import Resource, VaryingResource
from energiapy.components.process import Process, VaryingProcess
from energiapy.components.material import Material
from energiapy.components.location import Location
from energiapy.components.network import Network
from energiapy.components.scenario import Scenario
from energiapy.components.transport import Transport
from energiapy.components.result import Result 
from energiapy.utils.data_utils import get_data, make_henry_price_df, remove_outliers, load_results
from energiapy.model.formulate import formulate, Constraints, Objective
from energiapy.utils.nsrdb_utils import fetch_nsrdb_data
from energiapy.plot import plot_results, plot_scenario
from energiapy.plot.plot_results import CostY, CostX
from energiapy.model.solve import solve
from energiapy.aggregation.reduce_scenario import reduce_scenario, Clustermethod
from energiapy.aggregation.ahc import agg_hierarchial_elbow, IncludeAHC, Fit
from energiapy.aggregation.dtw import dynamic_warping, dynamic_warping_matrix, dynamic_warping_path, IncludeDTW
from energiapy.utils.data_utils import load_results
import matplotlib.pyplot as plt
from matplotlib import rc


**Import solar dni and wind speeds for Harris county**

In [23]:
weather20_df = pandas.read_csv('../data/ho_solar20.csv', index_col=0)
weather20_df.index = [i.split('+')[0] for i in weather20_df.index]
weather = weather20_df[~weather20_df.index.str.contains('02-29')] #remove leap years

**Demand data from ERCOT**

In [24]:
ercot20 = pandas.read_excel('../data/Native_Load_2020.xlsx')
ercot = ercot20[['COAST']]
ercot['index'] = weather20_df.index
ercot = ercot.set_index('index')
ercot = ercot[~ercot.index.str.contains('02-29')]

C:\Users\cacodcar\AppData\Local\Temp\ipykernel_15564\1843758898.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ercot['index'] = weather20_df.index


**Import natural gas prices from Henry Hub Price Index**  

In [25]:
#Actual temporal scale (daily)
ng_price20 = make_henry_price_df(
    file_name='../data/Henry_Hub_Natural_Gas_Spot_Price_Daily.csv', year=2020, stretch=False)
ng_price_df = ng_price20
ng_price_df['index'] = [i for i in weather.index][::24]
ng_price_df = ng_price_df.drop(columns= 'scales')
ng_price = ng_price_df.set_index('index')

$\textbf{Define temporal scale}$


In [26]:
scales = TemporalScale(discretization_list=[1, 365, 24], start_zero= 2020)

$\textbf{Declare constants for ease}$


In [27]:
bigM = 10**4  # very large number
smallM = 0.1
water_price = 31.70  # $/5000gallons
power_price = 8  # cents/kWh
ur_price = 42.70  # 250 Pfund U308 (Uranium)
A_f = 0.05  # annualization factor
# CO2_res = 0.2
pv_start = 0
ake_start = 0
smrh_start = 0
smr_start = 0
asmr_start = 0

$\textbf{Declare resources}$

In [28]:
Charge = Resource(name='Charge', sell=False,
                  store_max=100, basis='MW', label='Battery energy', block='energystorage')

Solar = Resource(
    name='Solar', cons_max=bigM, basis='MW', label='Solar Power', block='energyfeedstock')

Wind = Resource(name='Wind', cons_max= bigM, basis='MW', label='Wind Power', block='energyfeedstock')

H2O = Resource(name='H2O', cons_max=10**10, price=water_price/(5000*3.7854), basis='kg', label='Water', block='Resource')

CO2 = Resource(name='CO2', basis='kg', label='Carbon dioxide', block='Resource')

Power = Resource(name='Power', basis='MW', sell= True, demand = True, label='Renewable power generated', block='Resource')

Mile = Resource(name = 'Mile', basis = 'miles', sell = True, demand  = True, label = 'miles driven')

H2_L = Resource(name='H2_L', store_max=10**10, revenue=2,
                 basis='kg', label='Hydrogen - Geological', block='resourcestorage')

H2_C = Resource(name='H2_C', store_max= 10**10, loss=0.025/24, revenue=2,
    basis='kg', label='Hydrogen - Local Cryo', block='resourcestorage')
# Geothermal_energy= Resource(name='Geothermal', store_max=10**10, sell=True, basis='MW',  label='Geothermal Storage', block='resourcestorage')
# Biogas = Resource(name = 'Biogass', basis = 'kg', cons_max = 10**5, label = 'Biogas consumed from outside the system' )
# Biomass = Resource(name = 'Biomass', basis = 'kg', cons_max = 10**5, label = 'Biomass consumed from outside the system' )


$\textbf{Declare Materials}$


In [29]:
LiR = Material(name='LiR', gwp=1.484, resource_cons = {H2O: 2273}, toxicity=793, basis= 'kg', label='Rock-based Lithium', \
    citation= 'Nelson Bunyui Manjong (2021), httoxicitys://www.tcc.fl.edu/media/divisions/academic-affairs/academic-enrichment/urc/poster-abstracts/Xanders_Madison_Poster_URS.pdf') #gwp=(0.216,0.314)
LiB = Material(name='LiB', gwp=0.031, toxicity=793, basis= 'kg', label='Brine-based Lithium', citation= 'Nelson Bunyui Manjong (2021)') #gwp=(0.289,0.499)
Ni = Material(name='Ni', gwp=7.64, resource_cons = {H2O: 80}, toxicity=67, basis= 'kg', label='Nickel', citation= 'Mark Mistry (2016), Sustainable water use in minerals and metal production')
Co = Material(name='Co', gwp=11.73, toxicity=325, basis= 'kg', label='Cobalt', citation= 'Farjana et al. 2019a, b')
# Steel = Material(name='Steel', gwp=(0.8,1.4), resource_cons = {H2O: 3.94}, toxicity=40, basis= 'kg', label='Steel', citation= 'Kim R.Bawden (2016), Anlong Li (2020)') #toxicity of iron instead
Steel = Material(name='Steel', gwp=0.8, resource_cons = {H2O: 3.94}, toxicity=40, basis= 'kg', label='Steel', citation= 'Kim R.Bawden (2016), Anlong Li (2020)') #toxicity of iron instead
Na = Material(name='Na', gwp=0.5, resource_cons = {H2O: 3}, toxicity=20, basis= 'kg', label='Sodium')

# Fe = Material(name='Steel', gwp=(0.8,1.4), resource_cons = {H2O: 3.94}, toxicity=40, basis= 'kg', label='Iron', citation= 'Kim R.Bawden (2016), Anlong Li (2020)')
Fe = Material(name='Steel', gwp=0.8, resource_cons = {H2O: 3.94}, toxicity=40, basis= 'kg', label='Iron', citation= 'Kim R.Bawden (2016), Anlong Li (2020)')
CuP = Material(name='Cu', gwp=2.5, resource_cons = {H2O: 30}, toxicity=2500, basis= 'kg', label='Copper Pyro', citation= 'T.E. Norgate (2007), Sustainable water use in minerals and metal production')
CuH = Material(name='Cu', gwp=6, resource_cons = {H2O: 40}, toxicity=2500, basis= 'kg', label='Copper Hyro', citation= 'T.E. Norgate (2007), Sustainable water use in minerals and metal production')
PbB = Material(name='PbB', gwp=1.5, resource_cons = {H2O: 15}, toxicity=450, basis= 'kg', label='Lead BF', citation= 'T.E. Norgate (2007), Sustainable water use in minerals and metal production')
PbI = Material(name='PbI', gwp=2.5, resource_cons = {H2O: 25}, toxicity=450, basis= 'kg', label='Lead ISF', citation= 'T.E. Norgate (2007), Sustainable water use in minerals and metal production')
ZnE = Material(name='ZnE', gwp=5, resource_cons = {H2O: 30}, toxicity=57, basis= 'kg', label='Zn Elec', citation= 'T.E. Norgate (2007), Sustainable water use in minerals and metal production')
ZnI = Material(name='ZnI', gwp=3.5, resource_cons = {H2O: 25}, toxicity=57, basis= 'kg', label='Zn ISF', citation= 'T.E. Norgate (2007), Sustainable water use in minerals and metal production')
Pd = Material(name='Pd', gwp=3880, resource_cons = {H2O: 210713}, toxicity=3, basis= 'kg', label='Palladium', citation= 'Philip Nuss (2014), Simon Meißner (2021)')
Rd = Material(name='Rd', gwp=35100, toxicity=15, basis= 'kg', label='Rhodium', citation= 'Philip Nuss (2014)')
Pt = Material(name='Pd', gwp=12500, resource_cons = {H2O: 31349}, toxicity=5000, basis= 'kg', label='Platinum', citation= 'Philip Nuss (2014), Simon Meißner (2021)')
Y = Material(name='Y', gwp=15.1, toxicity=400, basis= 'kg', label='Yttrium', citation= 'Philip Nuss (2014)')
Al = Material(name='Al', gwp=8.2, resource_cons = {H2O: 147}, toxicity=3632, basis= 'kg', label='Aluminium', citation= 'Philip Nuss (2014), Water requirements of the aluminum industry Water Supply Paper 1330-C')
Mg = Material(name='Mg', gwp=9.6, toxicity=200, basis= 'kg', label='Magnesium', citation= 'Philip Nuss (2014)')
PP = Material(name='PP', gwp=1.586, basis= 'kg', label='Polypropylene', citation= 'Greenhouse gas emissions and natural capital implications of plastics (including biobased plastics)')
HDPE = Material(name='HDPE', gwp=1.98,toxicity=5000, basis= 'kg', label='HD polyethylene', citation= 'Greenhouse gas emissions and natural capital implications of plastics (including biobased plastics)')
LDPE = Material(name='LDPE', gwp=1.93, toxicity=2000, basis= 'kg', label='LD polyethylene', citation= 'Greenhouse gas emissions and natural capital implications of plastics (including biobased plastics)')
PVC = Material(name='PVC', gwp=2.51, basis= 'kg', label='Polyvinylchloride', citation= 'Greenhouse gas emissions and natural capital implications of plastics (including biobased plastics)')
Rubber = Material(name='Rubber', gwp=6.4, toxicity=330, basis= 'kg', label='Rubber', citation= 'GREENING OF INDUSTRY NETWORK (GIN) 2010: CLIMATE CHANGE AND GREEN GROWTH: INNOVATING FOR SUSTAINABILITY ')
PTFE = Material(name='PTFE', gwp=9.6, toxicity=11280, basis= 'kg', label='Poly tetra fluoroethylene', citation='httoxicitys://shamrocktechnologies.com/co2-emissions/')
# Epoxy = Material(name='Epoxy', gwp=(4.7, 8.1), toxicity=2000 , basis= 'kg', label='Epoxy resin', citation= 'Bricout et al. (2017)' )
Epoxy = Material(name='Epoxy', gwp=4.7, toxicity=2000 , basis= 'kg', label='Epoxy resin', citation= 'Bricout et al. (2017)' )

Polyester = Material(name='Polyester', gwp=4.32, toxicity=2500, basis= 'kg', label='Polyester resin', citation= ' Rietveld and Hegger (2014)' )
Vinyl_easter = Material(name='vinyl ester', gwp=5.87, toxicity=1000, basis= 'kg', label='vinyl ester resin', citation= ' Rietveld and Hegger (2014) ' ) #Assume toxicity
Glass_fibre = Material(name='glass fibre', gwp=2.5, toxicity=1000, basis= 'kg', label='Glass fibre resin', citation= ' Bachmann et al. (2017)' ) #Assume toxicity
Mn = Material(name='Mn', gwp=4.51, toxicity=1484, basis= 'kg', label='Manganese', citation= 'Nelson Bunyui Manjong (2021)') #inventory data #9.6, 2.6,7.9
Monocrystalline = Material(name='Mc', gwp=20, resource_cons = {H2O: 1.5}, toxicity=40.74, basis= 'kg', label='Mono', 
citation= 'Environmental impact assessment of monocrystalline silicon solar photovoltaic cell production: a case study in China (toxicity)')
Polycrystalline = Material(name='Pc', gwp=26 , resource_cons = {H2O: 1}, toxicity=15, basis= 'kg', label='Poly', citation= 'Environmental Economic Impact Assessments of a ... - MDPI (gwp and toxicity)')
#1,4-DB-eq
CdTE = Material(name='CdTe', gwp=40, resource_cons = {H2O: 0.25}, toxicity=5, basis= 'kg', label='C', citation= 'Investigation of life cycle CO2 emissions of the polycrystalline and cadmium telluride PV panels')




$\textbf{Declare processes}$

In [30]:

#THREE WIND FARMS - COMPLETELY DIFFERENT 
WF1 = Process(name='WF1', conversion={Wind: -5, Power: 1, H2O: -1},
             capex=1377, fopex=2117, vopex=300,
             prod_max=100, gwp=42700, land=1800000, trl='nrel', block='power_generation', material_cons = {Steel: 292, Fe:44, CuP: 4, Al: 4, Epoxy: 16, Polyester: 16, \
             Vinyl_easter: 12, Glass_fibre: 12},
             label='Wind mill array', citation='Use windtoolkit conversion') 

WF2 = Process(name='WF2', conversion={Wind: -3.33, Power: 1, H2O: -1}, 
               capex=5737, fopex=8824, vopex=900,
               prod_max=100, gwp=52700, land=1400000, trl='nrel', block='power_generation', material_cons = {Steel: 316, Fe:20, CuP: 4, Al: 4, Epoxy: 16, Polyester: 16, \
                Vinyl_easter: 12, Glass_fibre: 12},
               label='Wind mill array', citation='Use windtoolkit conversion, Techno-Economic Analysis and Modelling of the Feasibility of Wind Energy in Kuwait for land')

WF3 = Process(name='WF3', conversion={Wind: -2.5, Power: 1, H2O: -1},
               capex=567000, fopex=872046, vopex=90000,
               prod_max=100, gwp=62700, land=1000000, trl='nrel', block='power_generation', material_cons = {Steel: 330, Fe:85, CuP: 5, Al: 10, Epoxy: 20, Polyester: 20, \
                Vinyl_easter: 15, Glass_fibre: 15},
               label='Wind mill array', citation='Use windtoolkit conversion')

PV1 = Process(name='PV1', introduce=pv_start, conversion={Solar: -5, Power: 1, H2O: -0.0757}, material_cons = {Monocrystalline: 0.4}, 
              capex=990637, fopex=3354, vopex=4953,
             prod_max=100, gwp=46, land=13320/1800, trl='nrel', block='power_generation', \
                 label='Solar photovoltaics (PV) array', citation='Use pvlib conversion, \
                 Environmental Impacts of Photovoltaics: The Effects of Technological Improvements and Transfer of Manufacturing from Europe to China for gwp, \
                    Technoeconomic analysis of high-value, crystalline silicon photovoltaic module recycling processes for material,\
                    A Techno-Economic Feasibility Analysis of Mono-Si and Poly-Si Photovoltaic Systems in the Rooftop Area of Commercial Building under the Feed-In Tariff Scheme for money') 

PV2 = Process(name='PV2', introduce=pv_start, conversion={Solar: -6.67, Power: 1, H2O: -0.0857}, material_cons = {Polycrystalline: 0.36}, 
             prod_max=100, gwp=53, land=13320/1800, trl='nrel', block='power_generation', \
                 label='Solar photovoltaics (PV) array', citation='Use pvlib conversion, for gwp, for money, 10*0.036')

PV3 = Process(name='PV3', introduce=pv_start, conversion={Solar: -3.33, Power: 1, H2O: -0.0957}, material_cons = {CdTE: 0.02},
             prod_max=100, gwp=18, land=13320/1800, trl='nrel', block='power_generation', \
                 label='Solar photovoltaics (PV) array', citation='Use pvlib conversion, for gwp')

#Lead_a_d = Process(name='PbB_d',  conversion={Charge: -1.176, Power: 1}, capex=0.001, fopex=0.001, vopex=0, label='Lead-acid', citation= 'Florin, N. and Dominish, E. (2017), Zakeri 2015')
#Lead_a_c = Process(name='PbB_c',  conversion={Charge: 1, Power: -1}, trl='utility', capex=260000, fopex=5900, vopex=0, material_cons = {PbB: 20}, gwp=0.075, lifetime=(5,15), label='Lead-acid', citation= 'Florin, N. and Dominish, E. (2017), Zakeri 2015')

Na_S_c = Process(name='Na-S_c', conversion={Charge: 1, Power: -1},  trl='utility', lifetime=(10,15), material_cons = {Na: 40}, capex=1002181.8181818181, fopex=31432.7272727272, vopex=0, label='Sodium-Sulfur', citation= 'Florin, N. and Dominish, E. (2017), Zakeri 2015')

Na_S_d = Process(name='Na-S_d', conversion={Charge: -1.25, Power: 1}, label='Sodium-Sulfur', citation= 'Florin, N. and Dominish, E. (2017), Zakeri 2015')

#Lead_b_c = Process(name='PbI_c',  conversion={Charge: 1, Power: -1}, capex=300000, fopex=7900, vopex=0,trl='utility', material_cons = {PbI: 20}, gwp=0.075, lifetime=(5,15), label='Lead-acid', citation= 'Florin, N. and Dominish, E. (2017), Zakeri 2015')
#Lead_b_d = Process(name='PbI_d',  conversion={Charge: -1.196, Power: 1}, capex=0.001, fopex=0.001, vopex=0, label='Lead-acid', citation= 'Florin, N. and Dominish, E. (2017), Zakeri 2015')


Ni_Cd_c = Process(name='Ni-Cd_c', conversion={Charge: 1, Power: -1},trl='utility', material_cons = {Ni: 20}, capex=2002181.8181818181, fopex=71432.7272727272, vopex=0, gwp=0.08, lifetime=(10,20), label='Ni-Cd', citation= 'Florin, N. and Dominish, E. (2017), Zakeri 2015')
Ni_Cd_d = Process(name='Ni-Cd_D',  conversion={Charge: -1.54, Power: 1},label='Ni-Cd', citation= 'Florin, N. and Dominish, E. (2017), Zakeri 2015')

#DIFFERENT KINDS OF LITHIUM ION
LiI_a_c = Process(name='LiR_c', conversion={Charge: 1, Power: -1}, material_cons = {LiR: 20}, capex=1502181.8181818181, fopex=51432.7272727272, vopex=0, #cost = cost_dict['HO']['moderate']['LiI_c']['0'],\
    prod_max=100, trl='nrel', block='power_storage', label='Lithium-ion battery R', citation='Zakeri 2015')

LiI_a_d = Process(name='LiR_d', conversion={Charge: -1.1765, Power: 1}, capex=0.001, fopex=0.001, vopex=0, \
    
    prod_max=100, trl='discharge', block='power_storage', label='Lithium-ion battery discharge R', citation='Zakeri 2015')

LiI_b_c = Process(name='LiB_c', conversion={Charge: 1, Power: -1}, material_cons = {Na: 30}, capex=1302181.8181818181, fopex=41432.7272727272, vopex=0, #cost = cost_dict['HO']['moderate']['LiI_c']['0'],\
    prod_max=100, trl='nrel', block='power_storage', label='Lithium-ion battery B', citation='Zakeri 2015')

LiI_b_d = Process(name='LiB_d', conversion={Charge: -1.05, Power: 1}, capex=0.001, fopex=0.001, vopex=0, #cost = cost_dict['HO']['moderate']['LiI_c']['0'],\
    prod_max=100, trl='nrel', block='power_storage', label='Lithium-ion battery B', citation='Zakeri 2015')



#Vrb_c = Process(name='VRB_c', conversion={Charge: 1, Power: -1}, material_cons = {PbB: 20}, trl='utility', gwp=0.02, lifetime=(5,10), label='VRB', citation= 'Florin, N. and Dominish, E. (2017), Zakeri 2015')
#Vrb_d = Process(name='VRB_d',  conversion={Charge: -1.33, Power: 1}, label='VRB', citation= 'Florin, N. and Dominish, E. (2017), Zakeri 2015')




$\textbf{Declare location(s)}$


Scenario with all processes considered

In [31]:
process_set = {PV1, PV2, PV3, WF1, WF2, WF3, Na_S_c, Na_S_d,
               Ni_Cd_c, Ni_Cd_d, LiI_a_c, LiI_a_d, LiI_b_c, LiI_b_d}

HO = Location(name='HO', processes=process_set, demand_factor={Power: ercot},  capacity_factor={PV1: pandas.DataFrame(weather['dni']), WF1: pandas.DataFrame(weather['wind_speed'])}, scales=scales,
              label='Houston', demand_scale_level=2, capacity_scale_level=2, price_scale_level=1)

scenario0 = Scenario(name='shell', network=HO, scales=scales,  expenditure_scale_level=0, scheduling_scale_level=2,
                     network_scale_level=0, demand_scale_level=2, label='shell milp case study (HO)', demand= {HO: {Power: 100}})


milp0 = formulate(scenario=scenario0, constraints={
                  Constraints.COST, Constraints.INVENTORY, Constraints.PRODUCTION, Constraints.LAND, Constraints.RESOURCE_BALANCE, Constraints.EMISSION}, objective=Objective.COST)




constraint process capex
constraint process fopex
constraint process vopex
constraint process incidental
constraint location capex
constraint location fopex
constraint location vopex
constraint location incidental
constraint network capex
constraint network fopex
constraint network vopex
constraint network incidental
constraint global warming potential process
constraint global warming potential resource
constraint global warming potential material
constraint global warming potential location
constraint global warming potential network
constraint nameplate inventory
constraint storage max
constraint storage min
constraint production mode
constraint nameplate production
constraint production max
constraint production min
constraint land process
constraint land location
constraint land network
constraint land process cost
constraint land location cost
constraint land network cost
constraint inventory balance
constraint resource consumption
constraint resource purchase
constraint location

In [32]:
results0 = solve(scenario=scenario0, instance=milp0,
                 solver='gurobi', name=f"Material_case_study0")


Set parameter QCPDual to value 1
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: Intel(R) Core(TM) i7-6700 CPU @ 3.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 359357 rows, 315545 columns and 1112902 nonzeros
Model fingerprint: 0xc6a55ef6
Coefficient statistics:
  Matrix range     [1e-03, 2e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+01, 1e+10]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 262997 rows and 210417 columns
Presolve time: 0.46s
Presolved: 96360 rows, 105128 columns, 341639 nonzeros

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Ordering time: 0.03s

Barrier statistics:
 Dense cols : 8
 AA' NZ     : 2.365e+05
 Factor NZ  : 9.681e+05 (roughly 90 MB of memory)
 Factor Ops : 1.043e+07 (less than 1 second

In [33]:
process_set1 = {PV1, PV2, PV3, Na_S_c, Na_S_d, Ni_Cd_c, Ni_Cd_d, LiI_a_c, LiI_a_d, LiI_b_c, LiI_b_d}


HO1 = Location(name='HO1', processes= process_set1, demand_factor= {Power: ercot},  capacity_factor = {PV1: pandas.DataFrame(weather['dni']), WF1: pandas.DataFrame(weather['wind_speed'])}, scales=scales, \
        label='Houston', demand_scale_level=2, capacity_scale_level= 2, cost_scale_level= 1)

scenario1 = Scenario(name= 'shell1', network= HO1, scales= scales,  expenditure_scale_level= 0, scheduling_scale_level= 2, \
    network_scale_level= 0, demand_scale_level= 2, label= 'shell milp case study (HO)')

milp1 = formulate(scenario= scenario1, demand = 10, \
    constraints={Constraints.cost, Constraints.inventory, Constraints.production, Constraints.land, Constraints.resource_balance, Constraints.emission}, \
        objective= Objective.cost)

# results1 = solve(scenario = scenario1, instance= milp1, solver= 'gurobi', \
#         name=f"Material_case_study1")




TypeError: Location.__init__() got an unexpected keyword argument 'cost_scale_level'

In [ ]:
process_set2 = {WF1, WF2, WF3, Na_S_c, Na_S_d, Ni_Cd_c, Ni_Cd_d, LiI_a_c, LiI_a_d, LiI_b_c, LiI_b_d}

HO2 = Location(name='HO2', processes= process_set2, demand_factor= {Power: ercot}, \
     capacity_factor = {PV2: pandas.DataFrame(weather['dni']), WF2: pandas.DataFrame(weather['wind_speed'])}, scales=scales, \
        label='Houston', demand_scale_level=2, capacity_scale_level= 2, cost_scale_level= 1)

scenario2 = Scenario(name= 'shell2', network= HO2, scales= scales,  expenditure_scale_level= 2, scheduling_scale_level= 2, \
     network_scale_level= 0, demand_scale_level= 2, label= 'shell milp case study (HO)')

milp2 = formulate(scenario= scenario2, demand = 10, \
    constraints={Constraints.cost, Constraints.inventory, Constraints.production, Constraints.land, Constraints.resource_balance, Constraints.emission}, \
        objective= Objective.cost)

# results2 = solve(scenario = scenario2, instance= milp2, solver= 'gurobi', \
#         name=f"Material_case_study2")



In [ ]:

process_set3 = {WF1, WF2, WF3, PV1, PV2, PV3, Na_S_c, Na_S_d, Ni_Cd_c, Ni_Cd_d}

HO3 = Location(name='HO3', processes= process_set3, demand_factor= {Power: ercot}, \
     capacity_factor = {PV3: pandas.DataFrame(weather['dni']), WF3: pandas.DataFrame(weather['wind_speed'])}, scales=scales, \
        label='Houston', demand_scale_level=2, capacity_scale_level= 2, cost_scale_level= 1)

scenario3 = Scenario(name= 'shell3', network= HO3, scales= scales,  expenditure_scale_level= 2, scheduling_scale_level= 2, \
    network_scale_level= 0, demand_scale_level= 2, label= 'shell milp case study (HO)')

milp3 = formulate(scenario= scenario3, demand = 10, \
    constraints={Constraints.cost, Constraints.inventory, Constraints.production, Constraints.land, Constraints.resource_balance, Constraints.emission}, \
       objective= Objective.cost)

# results3 = solve(scenario = scenario3, instance= milp3, solver= 'gurobi', \
#         name=f"Material_case_study")

In [ ]:
scenario_dict = {
    0: {'scenario': scenario0, 'milp': milp0 },
    1: {'scenario': scenario1, 'milp': milp1 },
    2: {'scenario': scenario2, 'milp': milp2 },
    3: {'scenario': scenario3, 'milp': milp3 }}


In [ ]:
results_dict = {i: solve(scenario = scenario_dict[i]['scenario'], instance= scenario_dict[i]['milp'], solver= 'gurobi', \
        name=f"Material_case_study{i}") for i in range(4)}


In [ ]:
plot.schedule(location= 'HO', component = 'Charge', y_axis='Inv', results= results_dict[0])
plot.schedule(location= 'HO1', component = 'Charge', y_axis='Inv', results= results_dict[1])
plot.schedule(location= 'HO2', component = 'Charge', y_axis='Inv', results= results_dict[2])
plot.schedule(location= 'HO3', component = 'Charge', y_axis='Inv', results= results_dict[3])
#plot.schedule(location= 'HO3', component = 'PV3', y_axis='P', results= results_dict[3])
#plot.contribution(location= 'HO', y_axis = 'S_location', results= results_dict[0])
#plot.contribution(location= 'HO1', y_axis = 'S_location', results= results_dict[1])
#plot.contribution(location= 'HO2', y_axis = 'S_location', results= results_dict[2])
#plot.contribution(location= 'HO3', y_axis = 'S_location', results= results_dict[3])
plot.contribution(location= 'HO', y_axis = 'Cap_P', results= results_dict[0])
plot.contribution(location= 'HO1', y_axis = 'Cap_P', results= results_dict[1])
plot.contribution(location= 'HO2', y_axis = 'Cap_P', results= results_dict[2])
plot.contribution(location= 'HO3', y_axis = 'Cap_P', results= results_dict[3])
plot.capacity_utilization(location= 'HO', process = 'WF1', results= results_dict[0])
plot.capacity_utilization(location= 'HO1', process = 'PV3', results= results_dict[1])
plot.capacity_utilization(location= 'HO2', process = 'WF1', results= results_dict[2])
plot.capacity_utilization(location= 'HO3', process = 'PV3', results= results_dict[3])
plot.capacity_factor(process=WF1, location= HO) #same for 3 WF
plot.capacity_factor(process=PV1, location= HO, color='orange')
#plot.schedule(location= 'HO1', component = 'HV', y_axis='Inv', results= results1)
#plot.schedule(location= 'HO', component = 'H2_L', y_axis='Inv', results= results0) #same
#plot.schedule(location= 'HO1', component = 'H2_L', y_axis='Inv', results= results1)
#plot.schedule(location= 'HO2', component = 'H2_L', y_axis='Inv', results= results2)
#plot.schedule(location= 'HO3', component = 'H2_L', y_axis='Inv', results= results3)
#plot.demand_factor(location=HO1, resource= Mile) #same for 3 loc


In [ ]:
help(Constraints)

In [ ]:
from energiapy.model.constraints.emission import global_warming_potential_network_reduction_constraint

In [ ]:
results_dict[0].output['global_warming_potential_network'], results_dict[1].output['global_warming_potential_network'], results_dict[2].output['global_warming_potential_network'], results_dict[3].output['global_warming_potential_network']

In [ ]:
reduction_list = [(i+1)*15 for i in range(5)]

In [ ]:
def generate_milp(scenario):
    return formulate(scenario= scenario0, demand = 10, \
        constraints={Constraints.cost, Constraints.inventory, Constraints.production, Constraints.land, Constraints.resource_balance, Constraints.emission}, \
            objective= Objective.cost)


In [ ]:

def gwp_reduced_milp(scenario, gwp_reduction_pct, gwp, milp):
    return global_warming_potential_network_reduction_constraint(instance = generate_milp(scenario= scenario),\
     network_scale_level =0, gwp_reduction_pct = gwp_reduction_pct, gwp = gwp)
    


In [ ]:
instance_dict = {i: gwp_reduced_milp(scenario0, reduction_list[i], 420022, generate_milp(scenario= scenario0))  for i in range(5)}

In [ ]:
results_dict2 = {i:solve(scenario = scenario0, instance= instance_dict[i], solver= 'gurobi', \
        name=f"Material_case_study_red{i}") for i in range(5)}


In [ ]:
def gwp_reduce_solve(gwp_reduction_pct, gwp):
    milp = formulate(scenario= scenario0, gwp_reduction_pct= gwp_reduction_pct, gwp= gwp, constraints = {Constraints.cost, Constraints.inventory, Constraints.production, Constraints.land, Constraints.resource_balance, Constraints.emission}, objective = Objective.cost )
    results = solve(scenario = scenario0, instance= milp, solver= 'gurobi', \
        name=f"Material_case_study")
    return results

results_dict = {i: gwp_reduce_solve(gwp_reduction_pct=2*i, gwp = 420022) for i in range(5)}

In [ ]:
results_dict

In [ ]:

# results_dict = {i: gwp_reduce_solve(gwp_reduction_pct=0, gwp = 718419.21, scenario = scenario_dict[i]['scenario'], milp = scenario_dict[i]['milp']) for i in range(3)}
# 

In [ ]:
plot.contribution(location= 'HO1', y_axis = 'S_location', results= results_dict[0])

In [ ]:
# plot.capacity_factor(location= HO, process= PV, color= 'orange')
# plot.capacity_factor(location= HO, process= WF1, color= 'blue')
# plot.cost_factor (location= HO, resource= CH4, color= 'red')
# plot.demand_factor (location= HO, resource= Mile)

In [ ]:


#scenario = Scenario(name= 'shell', network= HO, scales= scales,  expenditure_scale_level= 2, scheduling_scale_level= 2, \
    #network_scale_level= 0, demand_scale_level= 2, label= 'shell milp case study (HO)')

$\textbf{Plot varying data input}$

In [ ]:
# plot.capacity_factor(location= HO_reduced, process= PV, color= 'orange')
# plot.capacity_factor(location= HO_reduced, process= WF, color= 'blue')
# plot.cost_factor (location= HO_reduced, resource= CH4, color= 'red')
# plot.demand_factor (location= HO_reduced, resource= Mile)

$\textbf{Formulate model}$

A pyomo instance is formulated from the scenario

Concises sets and corresponding variables are declared.

Corresponding constraints are generated based on the nature of model chosen

In the presented example, a MILP is formulated



In [ ]:
results_dict[0].output['Capex_location'], results_dict[1].output['Capex_location'], results_dict[2].output['Capex_location']

In [ ]:
results_dict[0].output['Vopex_location'], results_dict[1].output['Vopex_location'], results_dict[2].output['Vopex_location']


In [ ]:
results_dict[0].output['Fopex_location'], results_dict[1].output['Fopex_location'], results_dict[2].output['Fopex_location']


In [ ]:
results_dict[0].output['Cap_P'], results_dict[1].output['Cap_P'], results_dict[2].output['Cap_P']


In [ ]:
results_dict[0].output['X_P'], results_dict[1].output['X_P'], results_dict[2].output['X_P'], results_dict[3].output['X_P']

In [ ]:
results_dict[0].output['global_warming_potential_resource'], results_dict[1].output['global_warming_potential_resource'], results_dict[2].output['global_warming_potential_resource']


In [ ]:
results_dict[0].output['global_warming_potential_network'], results_dict[1].output['global_warming_potential_network'], results_dict[2].output['global_warming_potential_network'], results_dict[3].output['global_warming_potential_network']



## Results

In [ ]:
import numpy as np
bars2 = [420022, 164165, 420022, 188000]
barWidth = 0.3
 
# Choose the height of the blue bars

 
# Choose the height of the cyan bars

 
 
# The x position of bars
r1 = np.arange(len(bars2))
r2 = [x + barWidth for x in r1]
 
# Create blue bars

fig, ax = plt.subplots(figsize = (9,6))
# Create cyan bars
ax.bar(r2, bars2, width = barWidth, color = 'green', edgecolor = 'black', capsize=7, label='GWP')

# general layout
ax.set_xticks([r + barWidth for r in range(len(bars2))], ['WFs and PVs', 'PVs', 'WFs', 'PVs excluding Li-ion battery'], fontsize = 14)
plt.yticks(fontsize = 14)
ax.set_ylabel('GWP/kg CO2eq/MW', fontsize = 16)
ax.set_xlabel('Technology pathways', fontsize =16)
plt.legend()
plt.grid(alpha = 0.4)
plt.title('GWP comparison of different technology pathways', fontsize= 18)
# Show graphic
plt.show()

In [ ]:
import numpy as np
bars2 = [1.577e+07, 1.298e+08, 1.577e+07, 1.303e+08]
barWidth = 0.3
 
# Choose the height of the blue bars

 
# Choose the height of the cyan bars

 
 
# The x position of bars
r1 = np.arange(len(bars2))
r2 = [x + barWidth for x in r1]
 
# Create blue bars

fig, ax = plt.subplots(figsize = (9,6))
# Create cyan bars
ax.bar(r2, bars2, width = barWidth, color = 'cyan', edgecolor = 'black', capsize=7, label='cost')

# general layout
ax.set_xticks([r + barWidth for r in range(len(bars2))], ['WFs and PVs', 'PVs', 'WFs', 'PVs excluding Li-ion battery'], fontsize = 14)
plt.yticks(fontsize = 14)
ax.set_ylabel('Cost/$', fontsize = 16)
ax.set_xlabel('Technology pathways', fontsize =16)
plt.legend()
plt.grid(alpha = 0.4)
plt.title('Cost comparison of different technology pathways', fontsize= 18)
# Show graphic
plt.show()

In [ ]:
for i in range(4):
    print(f"============{i}=============")
    
    print(results_dict[i].output['global_warming_potential_process'])
    print(results_dict[i].output['global_warming_potential_material'])
    print(results_dict[i].output['global_warming_potential_resource'])



In [ ]:
156., 92

# libraries
import numpy as np
import matplotlib.pyplot as plt
 
# width of the bars
barWidth = 0.3
 
# Choose the height of the blue bars
bars1 = [420022, 507153, 164000]
 
# Choose the height of the cyan bars

 
 
# The x position of bars
r1 = np.arange(len(bars1))
r2 = [x + barWidth for x in r1]
 
# Create blue bars
plt.bar(r1, bars1, width = barWidth, color = 'blue', edgecolor = 'black', capsize=7, label='GWP')
 
# Create cyan bars

 
# general layout
plt.xticks([r + barWidth for r in range(len(bars1))], ['WF A and PV A', 'WF B and PV B', 'WF C and PV C'])
plt.ylabel('GWP')
plt.legend()
 
# Show graphic
plt.show()





# process_set = {LiI_c, LiI_d, CAES_c, CAES_d, PSH_c, PSH_d, WF, PV, AKE, SMRH, SMR, ASMR, H2_C_d, \
#     H2_C_d, H2_L_c, H2_L_d, DAC, EOR, AQoff_SMR, EV, HV, MV, Na_S_c, Lead_a_c, Lead_b_c, Vrb_c,  \
#         Ni_Cd_c, Na_S_d, Lead_a_d, Lead_b_d, Vrb_d, Ni_Cd_d, CO2_based_methanol, Biogas_plant, Biomass}
#  HV, MV,


# process_set1 = {LiI_c, LiI_d, DAC, WF1, PV, AKE, H2_L_c, H2_L_d, EV, Na_S_c, Lead_a_c, Lead_b_c, Vrb_c,  \
#         Ni_Cd_c, Na_S_d, Lead_a_d, Lead_b_d, Vrb_d, Ni_Cd_d, CO2_based_methanol, Biogas_plant, Biomass} 


# process_set1 = {LiI_c, LiI_d, DAC, WF1, PV, AKE, H2_L_c, H2_L_d, HV, Na_S_c, Lead_a_c, Lead_b_c, Vrb_c,  \
#         Ni_Cd_c, Na_S_d, Lead_a_d, Lead_b_d, Vrb_d, Ni_Cd_d, CO2_based_methanol, Biogas_plant, Biomass} 

# CO2_based_methanol = Process(name='methanol', conversion = {CO2: -1, CH3OH: 1}, trl='pilot', gwp=-1.7, label='CO2 CH3OH', citation= 'muller (2020)')
# Biogas_plant = Process(name='Biogas', trl='pilot', conversion = {Biogas:-1, Power: 1}, gwp=-0.42, lifetime=20, label='Biogas', citation= 'Economic and Global Warming Potential Assessment of Flexible Power Generation with Biogas Plants, Life Cycle Environmental Impacts of Electricity from Biogas Produced by Anaerobic Digestion')
# Biomass = Process(name='Biomass', trl='pilot', conversion = {Biomass: -1, Power: 1}, gwp=0.13, label='Biomass', citation= 'Analysis of the Global Warming Potential of Biogenic CO2 Emission in Life Cycle Assessments') #gwp=0.049 nrel

# CO2_based_methanol = Process(name='methanol', conversion = {CO2: -1, CH3OH: 1}, trl='pilot', gwp=(-1.7,9.7), label='CO2 CH3OH', citation= 'muller (2020)')
# Biogas_plant = Process(name='Biogas', trl='pilot', conversion = {Biogas:-1, Power: 1}, gwp=(-0.42, 0.06), lifetime=20, label='Biogas', citation= 'Economic and Global Warming Potential Assessment of Flexible Power Generation with Biogas Plants, Life Cycle Environmental Impacts of Electricity from Biogas Produced by Anaerobic Digestion')
# Biomass = Process(name='Biomass', trl='pilot', conversion = {Biomass: -1, Power: 1}, gwp=(0.13,0.32), label='Biomass', citation= 'Analysis of the Global Warming Potential of Biogenic CO2 Emission in Life Cycle Assessments') #gwp=0.049 nrel
